In [5]:
import nltk
import json

In [ ]:
# sentence = "A URINAL IN A PUBLIC RESTROOM NEAR A WOODEN TABLE"
# sentence = "A PERSON WITH GREEN SKIS SKIING DOWN A BIG HILL"
write_path_format = "/data/user_data/sbharad2/SpeechCLIP/data/coco/SpokenCOCO/SpokenCOCO_{split}_with_nouns.json"
read_path_format = (
    "/data/user_data/sbharad2/SpeechCLIP/data/coco/SpokenCOCO/SpokenCOCO_{split}.json"
)


def get_all_nouns(sentence):
    nouns = []
    for i, (word, postag) in enumerate(nltk.pos_tag(sentence.lower().split())):
        if postag == "NN":
            nouns.append(word)
    return nouns

In [ ]:
# data = None

# for split in ["train", "val"]:
#     print("Preparing data for split:", split)
#     data = None
#     with open(read_path_format.format(split=split), "r") as f:
#         data = json.load(f)["data"]
#         print("Total number of images:", len(data))
#         i = 0
#         for datum in data:
#             image_path = datum["image"]
#             for caption in datum["captions"]:
#                 caption["nltk_nouns"] = get_all_nouns(caption["text"])
#             if i % 10000 == 0:
#                 print(i)
#             i += 1
#     print("Writing to:", write_path_format.format(split=split))
#     with open(write_path_format.format(split=split), "w") as f:
#         json.dump({"data": data}, f)

In [ ]:
from collections import defaultdict

all_nouns_val = defaultdict(int)

with open(write_path_format.format(split="val"), "r") as f:
    data = json.load(f)["data"]

for datum in data:
    try:
        for cap in datum["captions"]:
            for nn in cap["nltk_nouns"]:
                if nn in all_nouns:
                    all_nouns_val[nn] += 1
                else:
                    all_nouns_val[nn] = 1
    except:
        print(datum)

print(len(all_nouns_val), "labels in validation set")

3615

In [22]:
from collections import defaultdict

all_nouns = defaultdict(int)

with open(write_path_format.format(split="train"), "r") as f:
    data = json.load(f)["data"]

removed = 0
total_caps = 0
modified_data = []
for datum in data:
    modified_datum = datum
    modified_caption = []
    for caption in datum["captions"]:
        total_caps += 1
        caption["target"] = [w for w in caption["nltk_nouns"] if w in all_nouns_val]
        if len(caption["target"]) == 0:
            removed += 1
        else:
            # only add this caption if it has a target noun
            modified_caption.append(caption)
    if len(modified_caption) > 0:
        # only add this image if it has at least one caption with a target noun
        modified_datum["captions"] = modified_caption
        modified_data.append(modified_datum)

new_write_path_train = "/data/user_data/sbharad2/SpeechCLIP/data/coco/SpokenCOCO/SpokenCOCO_{split}_with_nouns_filtered.json"
with open(new_write_path_train.format(split="train"), "w") as f:
    json.dump({"data": modified_data}, f)

print(
    "Total removed:",
    removed,
    "Total captions:",
    total_caps,
    "Total remaining:",
    total_caps - removed,
)

Total removed: 6108 Total captions: 592187 Total remaining: 586079


In [ ]:
all_nouns = {
    k: v for k, v in sorted(all_nouns.items(), key=lambda item: item[1], reverse=True)
}